In [1]:
# Cell 0: 모듈 import를 위한 경로 설정
import os, sys
sys.path.append(os.path.abspath(".."))  # shared, models 디렉토리 접근 가능하도록 경로 추가


In [2]:
# Cell 1: 환경 확인
import torch

print(f"✅ PyTorch version: {torch.__version__}")
print(f"🚀 GPU available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"🧠 GPU name: {torch.cuda.get_device_name(0)}")
    
    total_memory = torch.cuda.get_device_properties(device).total_memory / 1024**3  # GiB
    reserved_memory = torch.cuda.memory_reserved(device) / 1024**3  # GiB
    allocated_memory = torch.cuda.memory_allocated(device) / 1024**3  # GiB
    free_memory = reserved_memory - allocated_memory  # GiB

    print(f"💾 Total memory: {total_memory:.2f} GiB")
    print(f"📦 Reserved memory: {reserved_memory:.2f} GiB")
    print(f"📈 Allocated memory: {allocated_memory:.2f} GiB")
    print(f"🟢 Free memory in reserved: {free_memory:.2f} GiB")


✅ PyTorch version: 2.6.0+cu124
🚀 GPU available: True
🧠 GPU name: Quadro RTX 5000
💾 Total memory: 15.73 GiB
📦 Reserved memory: 0.00 GiB
📈 Allocated memory: 0.00 GiB
🟢 Free memory in reserved: 0.00 GiB


In [3]:
# Cell 2: 데이터셋 로딩
from torch.utils.data import DataLoader
from shared.data_loader import HDF5Dataset
import os

input_dir = "/caefs/data/IllustrisTNG/subcube/input"
output_dir = "/caefs/data/IllustrisTNG/subcube/output"

input_files = sorted([os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".h5")])
output_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".h5")])

dataset = HDF5Dataset(input_files, output_files)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

x, y = next(iter(loader))
print(f"✅ Sample loaded: input shape = {x.shape}, output shape = {y.shape}")


2025-06-13 14:02:56,755 | INFO | data_loader | 🔍 Initializing dataset with 9 file pairs.
2025-06-13 14:02:56,775 | INFO | data_loader | 📦 Total samples across all files: 884736


✅ Sample loaded: input shape = torch.Size([2, 1, 60, 60, 60]), output shape = torch.Size([2, 1, 60, 60, 60])


In [4]:
from models.fno.model import FNO
import torch
from torchinfo import summary

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# FNO 모델 초기화
model = FNO(
    in_channels=1,
    out_channels=1,
    modes1=16,
    modes2=16,
    modes3=16,
    width=128,
    lifting_channels=64,
    add_grid=True
).to(device)

model.train()
print("✅ FNO model loaded and set to training mode.")

2025-06-13 14:05:36,832 | INFO | models.fno.model | ✅ FNO model initialized successfully.
2025-06-13 14:05:36,939 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([2, 1, 60, 60, 60])
2025-06-13 14:05:36,940 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:36,944 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:36,953 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([2, 4, 60, 60, 60])


✅ FNO model loaded and set to training mode.


2025-06-13 14:05:37,277 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:37,280 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:37,283 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:37,286 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:37,633 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([2, 1, 60, 60, 60])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
FNO                                      [2, 1, 60, 60, 60]        [2, 1, 60, 60, 60]        --                        --
├─Linear: 1-1                            [432000, 4]               [432000, 64]              320                       --
├─Linear: 1-2                            [432000, 64]              [432000, 128]             8,320                     --
├─ModuleList: 1-3                        --                        --                        --                        --
│    └─SpectralConvolution: 2-1          [2, 128, 60, 60, 60]      [2, 128, 60, 60, 60]      70,528                    --
│    └─SpectralConvolution: 2-2          [2, 128, 60, 60, 60]      [2, 128, 60, 60, 60]      70,528                    --
│    └─SpectralConvolution: 2-3          [2, 128, 60, 60, 60]      [2, 128, 60, 60, 60]      70,528                    --
│    └─Spectra

In [5]:
summary(model, input_size=(2, 1, 60, 60, 60),
        col_names=["input_size", "output_size", "num_params", "kernel_size"])


2025-06-13 14:05:37,646 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([2, 1, 60, 60, 60])
2025-06-13 14:05:37,647 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:37,648 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:37,649 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([2, 4, 60, 60, 60])
2025-06-13 14:05:37,651 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:37,654 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:37,656 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:37,658 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:37,659 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([2, 1, 60, 60, 60])


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
FNO                                      [2, 1, 60, 60, 60]        [2, 1, 60, 60, 60]        --                        --
├─Linear: 1-1                            [432000, 4]               [432000, 64]              320                       --
├─Linear: 1-2                            [432000, 64]              [432000, 128]             8,320                     --
├─ModuleList: 1-3                        --                        --                        --                        --
│    └─SpectralConvolution: 2-1          [2, 128, 60, 60, 60]      [2, 128, 60, 60, 60]      70,528                    --
│    └─SpectralConvolution: 2-2          [2, 128, 60, 60, 60]      [2, 128, 60, 60, 60]      70,528                    --
│    └─SpectralConvolution: 2-3          [2, 128, 60, 60, 60]      [2, 128, 60, 60, 60]      70,528                    --
│    └─Spectra

In [6]:
def test_batch_size(batch_size):
    try:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        model.eval()
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                _ = model(x)
                print(f"✅ Success with batch_size={batch_size}")
                break
    except RuntimeError as e:
        print(f"❌ Failed with batch_size={batch_size}: {str(e).splitlines()[0]}")

for bs in [32, 16, 8, 4, 2, 1]:
    test_batch_size(bs)


2025-06-13 14:05:38,504 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([32, 1, 60, 60, 60])
2025-06-13 14:05:38,505 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:38,506 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:38,506 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([32, 4, 60, 60, 60])
2025-06-13 14:05:39,059 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([16, 1, 60, 60, 60])
2025-06-13 14:05:39,059 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:39,060 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:39,060 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([16, 4, 60, 60, 60])


❌ Failed with batch_size=32: CUDA out of memory. Tried to allocate 6.59 GiB. GPU 0 has a total capacity of 15.73 GiB of which 5.54 GiB is free. Process 12502 has 26.06 MiB memory in use. Including non-PyTorch memory, this process has 10.16 GiB memory in use. Of the allocated memory 9.95 GiB is allocated by PyTorch, and 63.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


2025-06-13 14:05:39,454 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([8, 1, 60, 60, 60])
2025-06-13 14:05:39,455 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:39,456 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:39,456 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([8, 4, 60, 60, 60])
2025-06-13 14:05:39,538 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:39,542 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:39,545 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:39,549 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:39,550 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([8, 1, 60, 60, 60])


❌ Failed with batch_size=16: CUDA out of memory. Tried to allocate 3.30 GiB. GPU 0 has a total capacity of 15.73 GiB of which 2.24 GiB is free. Process 12502 has 26.06 MiB memory in use. Including non-PyTorch memory, this process has 13.46 GiB memory in use. Of the allocated memory 11.57 GiB is allocated by PyTorch, and 1.74 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
✅ Success with batch_size=8


2025-06-13 14:05:40,339 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([4, 1, 60, 60, 60])
2025-06-13 14:05:40,340 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:40,341 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:40,341 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([4, 4, 60, 60, 60])
2025-06-13 14:05:40,388 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:40,392 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:40,395 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:40,398 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:40,399 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([4, 1, 60, 60, 60])


✅ Success with batch_size=4


2025-06-13 14:05:40,953 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([2, 1, 60, 60, 60])
2025-06-13 14:05:40,954 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:40,955 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:40,955 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([2, 4, 60, 60, 60])
2025-06-13 14:05:40,957 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:40,959 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:40,962 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:40,964 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:40,965 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([2, 1, 60, 60, 60])


✅ Success with batch_size=2


2025-06-13 14:05:41,429 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([1, 1, 60, 60, 60])
2025-06-13 14:05:41,430 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:41,430 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:41,431 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([1, 4, 60, 60, 60])
2025-06-13 14:05:41,450 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:41,454 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:41,458 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:41,461 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:41,462 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([1, 1, 60, 60, 60])


✅ Success with batch_size=1


In [7]:
from shared.losses import mse_loss, spectral_loss

loss_val = mse_loss(x.to(device), y.to(device))
print(f"✅ MSE Loss on sample batch: {loss_val.item():.4f}")


✅ MSE Loss on sample batch: 26.8519


In [8]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
print("✅ Optimizer and LR scheduler initialized.")


✅ Optimizer and LR scheduler initialized.


In [9]:
from tqdm import tqdm

model.train()
loader = DataLoader(dataset, batch_size=4, shuffle=True)
n_batch = 10

for epoch in range(3):
    total_loss = 0.0
    for i, (inputs, targets) in enumerate(tqdm(loader, desc=f"Epoch {epoch+1}")):
        if i >= n_batch:
            break
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = mse_loss(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    print(f"📉 Epoch {epoch+1} Loss: {total_loss / n_batch:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")


Epoch 1:   0%|          | 0/221184 [00:00<?, ?it/s]2025-06-13 14:05:44,961 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([4, 1, 60, 60, 60])
2025-06-13 14:05:44,966 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:05:44,967 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:05:44,968 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([4, 4, 60, 60, 60])
2025-06-13 14:05:44,973 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:05:44,976 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:05:44,979 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:05:44,982 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:05:44,984 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([4, 1, 60, 60, 60])
Epoch 1:   0%|          | 1/221184 [00:02<129:25:4

📉 Epoch 1 Loss: 1.0385 | LR: 9.76e-05


Epoch 2:   0%|          | 0/221184 [00:00<?, ?it/s]2025-06-13 14:06:02,537 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([4, 1, 60, 60, 60])
2025-06-13 14:06:02,539 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:06:02,540 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:06:02,540 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([4, 4, 60, 60, 60])
2025-06-13 14:06:02,545 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:06:02,548 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:06:02,551 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:06:02,554 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:06:02,555 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([4, 1, 60, 60, 60])
Epoch 2:   0%|          | 1/221184 [00:01<110:36:1

📉 Epoch 2 Loss: 1.0213 | LR: 9.05e-05


Epoch 3:   0%|          | 0/221184 [00:00<?, ?it/s]2025-06-13 14:06:19,346 | INFO | models.fno.model | 🚀 FNO forward pass started. Input shape: torch.Size([4, 1, 60, 60, 60])
2025-06-13 14:06:19,347 | INFO | models.fno.model | 🌐 Generating coordinate grid with shape: [60, 60, 60]
2025-06-13 14:06:19,349 | INFO | models.fno.model | ✅ Coordinate grid generated.
2025-06-13 14:06:19,350 | INFO | models.fno.model | 🔗 Added grid to input. New shape: torch.Size([4, 4, 60, 60, 60])
2025-06-13 14:06:19,355 | INFO | models.fno.model | 🔁 Passed through Fourier layer 1/4
2025-06-13 14:06:19,359 | INFO | models.fno.model | 🔁 Passed through Fourier layer 2/4
2025-06-13 14:06:19,363 | INFO | models.fno.model | 🔁 Passed through Fourier layer 3/4
2025-06-13 14:06:19,367 | INFO | models.fno.model | 🔁 Passed through Fourier layer 4/4
2025-06-13 14:06:19,368 | INFO | models.fno.model | ✅ Forward pass completed. Output shape: torch.Size([4, 1, 60, 60, 60])
Epoch 3:   0%|          | 1/221184 [00:01<103:29:1

📉 Epoch 3 Loss: 1.0006 | LR: 7.94e-05


In [10]:
save_path = "fno_test_model.pt"
torch.save(model.state_dict(), save_path)
print(f"✅ FNO model saved to {save_path}")

state_dict = torch.load(save_path, map_location='cpu')
print(f"🔍 저장된 키 개수: {len(state_dict)}")
print("예시 키:", list(state_dict.keys())[:5])


✅ FNO model saved to fno_test_model.pt
🔍 저장된 키 개수: 60
예시 키: ['p1.weight', 'p1.bias', 'p2.weight', 'p2.bias', 'fourier_blocks.0.core_real']
